In [1]:
import numpy as np
import pandas as pd
import os

In [16]:
data_directory = "../Data/Study/A/41/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [9]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [10]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [5]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [6]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.makedirs(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path, index=False)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [17]:
calculate_errors("error/recalibrated_calibration")

ssHeadConstrained_20230517151916_shifted27.csv
uncalibrated: 13.62977565373499
calibrated: 2.3413948569462963
cosine: 13.342408867303067

calibration_20230517152412_shifted6.csv
uncalibrated: 15.790041828179861
calibrated: 1.770323339825806
cosine: 15.3842031230286

wsBodyConstrained_20230517152542_shifted48.csv
uncalibrated: 24.463039842372595
calibrated: 25.74241187216683
cosine: 23.743369646586512

ssWalking_20230517152738_shifted48.csv
uncalibrated: 70.01055340351267
calibrated: 68.22328979253066
cosine: 86.1426221575955

hallway_20230517153005_shifted0.csv
uncalibrated: 35.51238301441948
calibrated: 25.082372108794786
cosine: 33.70786962818922

hallway_20230517151611_shifted27.csv
uncalibrated: 33.340674603554326
calibrated: 22.7641245404463
cosine: 32.10557178745135

ssHeadConstrained_20230517153316_shifted26.csv
uncalibrated: 13.960566633279266
calibrated: 2.306332064432914
cosine: 13.666165463820363

ssWalking_20230517151303_shifted0.csv
uncalibrated: 67.05224305336418
calibrat